In [54]:
from functions import energy, top_change, struct_dict_maker
import subprocess
import os

In [16]:
top = "/home/riley/Dispersion-Parameter-Optimization-D-POp-/top"
gcp_dict = dict({
    "def2-tzvp": "gcp(dft/tz)",
    "def2-svpd": "gcp(file)",
    "def2-svp" : "gcp(dft/svp)"
})

bottom = "/home/riley/Dispersion-Parameter-Optimization-D-POp-/bb"
au =  627.5095

In [6]:
with open(top, "r") as t:
    file = t.read().split()
anchor = "!"
#file
lib_params = [file[file.index(anchor) + i] for i in [1, 2, 3]]
lib_params

['tpss', 'def2-svp', 'gcp(dft/svp)']

In [12]:
def get_lib_params(file):
    lib_list = file.split()
    return [lib_list[lib_list.index(anchor) + i] for i in [1, 2, 3]]



In [10]:
print("This program constructs a library of interaction energies for only the DFT portion")
basis = input("What basis set do you want: ").lower()
functional = input("What functional do you want: ").lower()

This program constructs a library of interaction energies for only the DFT portion


What basis set do you want:  def2-svpd
What functional do you want:  blyp


In [14]:
with open(top, "r") as t:
    file = t.read()

lib_params = get_lib_params(file)
change_file = (file.replace(lib_params[0], functional)
                    .replace(lib_params[1], basis)
                    .replace(lib_params[2], gcp_dict[basis]))

change_file

'! blyp def2-svpd gcp(file)\n%pal \nNPROCS 6 \nend\n%maxcore 2000\n%method\n  D3A1 0.3065\n  D3S8 0.9147\n  D3A2 5.0570\nend\n* xyz 0 1\n'

In [24]:
def lib_constructor(basis_set, functional, struct_dict, name):
    with open(top, "r") as t:
        file = t.read()

    lib_params = get_lib_params(file)
    change_file = (file.replace(lib_params[0], functional)
                    .replace(lib_params[1], basis)
                    .replace(lib_params[2], gcp_dict[basis]))

    top_change(top, change_file)
    
    for struct in struct_dict.keys():

        energies = []

        for path in struct_dict[struct]:
            if "a_" in path.split("/")[-1]:
                change_data = change_data.replace("xyz 0", "xyz -1")
            elif "c_" in path.split("/")[-1]:
                change_data = change_data.replace("xyz 0", "xyz 1")

            
            subprocess.run(f"cat {top} {path} {bottom} > input", shell=True)
            energies.append(float(subprocess.check_output(f"/home/riley/bin/orca_6_0_0_shared_openmpi416/orca input| grep 'FINAL'| awk '{{print $5}}' ORS=' '",
                                                              shell=True,
                                                              executable="/bin/bash")
                                                              .decode()))

    
            subprocess.run("rm input*", shell=True)

        with open(name, 'a') as lib:
            lib.write(f"{struct} {au*(energies[0] - (energies[1]+energies[2]))}\n")
            
    top_change(top, file)

In [21]:
name = input("What name do you want for the library: ").lower()

What name do you want for the library:  abc


In [22]:
struct_dict = struct_dict_maker("all-s66")
struct_dict

{'x01': ['all-s66/x01dd.xyz', 'all-s66/x01m1.xyz', 'all-s66/x01m2.xyz'],
 'x02': ['all-s66/x02dd.xyz', 'all-s66/x02m1.xyz', 'all-s66/x02m2.xyz'],
 'x03': ['all-s66/x03dd.xyz', 'all-s66/x03m1.xyz', 'all-s66/x03m2.xyz'],
 'x04': ['all-s66/x04dd.xyz', 'all-s66/x04m1.xyz', 'all-s66/x04m2.xyz'],
 'x05': ['all-s66/x05dd.xyz', 'all-s66/x05m1.xyz', 'all-s66/x05m2.xyz'],
 'x06': ['all-s66/x06dd.xyz', 'all-s66/x06m1.xyz', 'all-s66/x06m2.xyz'],
 'x07': ['all-s66/x07dd.xyz', 'all-s66/x07m1.xyz', 'all-s66/x07m2.xyz'],
 'x08': ['all-s66/x08dd.xyz', 'all-s66/x08m1.xyz', 'all-s66/x08m2.xyz'],
 'x09': ['all-s66/x09dd.xyz', 'all-s66/x09m1.xyz', 'all-s66/x09m2.xyz'],
 'x10': ['all-s66/x10dd.xyz', 'all-s66/x10m1.xyz', 'all-s66/x10m2.xyz'],
 'x11': ['all-s66/x11dd.xyz', 'all-s66/x11m1.xyz', 'all-s66/x11m2.xyz'],
 'x12': ['all-s66/x12dd.xyz', 'all-s66/x12m1.xyz', 'all-s66/x12m2.xyz'],
 'x13': ['all-s66/x13dd.xyz', 'all-s66/x13m1.xyz', 'all-s66/x13m2.xyz'],
 'x14': ['all-s66/x14dd.xyz', 'all-s66/x14m1.xyz', 

In [27]:
lib_constructor(basis, functional, struct_dict, name) 

In [33]:
with open("a", "a") as a:
    a.write(f"{str([[1,2,3],[4,5,6]])}\n")

In [34]:
with open("a", "r") as a:
    fil = a.read()

In [35]:
fil

'simplex\n[1, 2, 3]\n[[1, 2, 3], [4, 5, 6]]\n[[2, 2, 3], [4, 5, 6]]\n'

In [44]:
ab = [x for x in fil.split('\n') if x]


In [45]:
ab

['simplex', '[1, 2, 3]', '[[1, 2, 3], [4, 5, 6]]', '[[2, 2, 3], [4, 5, 6]]']

In [46]:
import ast

In [47]:
abc = ast.literal_eval(ab[-1])
abc

[[2, 2, 3], [4, 5, 6]]

In [52]:
abc[0][1] + abc[0][2]

5

In [50]:
if os.path.isfile("checkpoint"):
    print("yes")

yes


In [55]:
def value_dict_maker(path):
    with open(path, "r") as f:
        value_list = [ x.split() for x in f.read().split("\n") if x]
    if "ref" in path.split("/")[-1]:
        return {x[0]: float(x[1]) for x in value_list}, {x[0]: float(x[2]) for x in value_list}
    return {x[0]: float(x[1]) for x in value_list}

In [63]:
a = value_dict_maker("s66_ref_energies_1")

In [67]:
len(a)

2

In [61]:
b

{}